# **Library**

In [ ]:
import random
import json
import numpy as np
from types import LambdaType
import matplotlib.pyplot as plt
import sympy as sp
import itertools as it

# **Regularization**

# **Simulator Class**

# Phase

In [ ]:
class Phase:
  def __init__(self, Model, DataSet):
    self._Model = Model
    self._Dataset = DataSet

    self._Metrics = dict()

    self._Metrics["Loss"] = []
    self._Metrics["Accuracy"] = []

  def GetMetrics(self):
    return self._Metrics

  def Work(self):
    raise NotImplemented

# Training Phase (Phase)

In [ ]:
class TrainPhase(Phase):
  def __init__(self, Model, TrainSet):
    super().__init__(Model, TrainSet)

  def Work(self,BatchDimension,**Hyperparameters):
    if(BatchDimension < 1 or BatchDimension > len(self._Dataset)): raise ValueError("batch dimension must be above 0 and lower or equal to the dataset dimension")
    Batches = BatchesExtraction(self._Dataset, BatchDimension)

    TotalLossValue = 0

    for Batch in Batches:
      #Do mean of all loss values
      LossValue = 0
      for r in Batch:
        InputVector=r[0]
        TargetValue=r[1]
        LossValue += self._Model.GetLossLambdaFunctionEvaluation()(self._Model.Predict(InputVector), TargetValue)

      LossValue /= len(Batch)

      #Add loss value batch to total loss value
      TotalLossValue += LossValue

      #Calculate gradient
      #DirectionLoss = self._Model.GradientDirectionLoss(LossValue)

      #Learn model
      self._Model.Learn(TotalLossValue)

    #Apply mean of all loss values
    self._Metrics["Loss"].append(TotalLossValue / len(Batches))

# Evaluation Phase (Phase)

In [ ]:
class EvaluationPhase(Phase):
  def __init__(self, Model, DataSet, n):
    super().__init__(Model, DataSet, n, lambda: True)

  def Work(self):

    Batches = BatchesExtraction(self._Dataset, 1)

    TotalLossValue = 0

    for Batch in Batches:

      #Do mean of all loss values
      LossValue = 0

      for r in Batch: LossValue += self._Model.GetLossLambdaFunctionEvaluation(self._Model.Predict(r[0]), r[1])
      LossValue /= len(r)

      #Add loss value batch to total loss value
      TotalLossValue += LossValue

    #Apply mean of all loss values
    self._Metrics["Loss"].append(TotalLossValue / len(Batches))

# Cross validation


In [ ]:
class CrossValidation:
  def __init__(self, Dataset, K):
    Data = random.sample(list(Dataset), len(Dataset))          #Shuffle data
    self.__Folds = np.split(Data, [len(Dataset)//K])           #Split data in k folds
    del Data                                                   #It is not necessary in memory anymore

  def GetFoldsNum(self):
    return len(self.__Folds)

  def GetKFold(self, K):
    ListFolds = list(self.__Folds)
    ValidationSet = np.array(ListFolds[K])
    del ListFolds[K]
    TrainingSet = np.concatenate(ListFolds)
    return np.array(TrainingSet, dtype=object), np.array(ValidationSet, dtype=object)


In [ ]:
Dataset = [[[11, 21, 31, 41], [51, 61]],
           [[12, 22, 32, 42], [52, 62]],
           [[13, 23, 33, 43], [53, 63]],
           [[14, 24, 34, 44], [54, 64]],
           [[15, 25, 35, 45], [55, 65]],
           [[16, 26, 36, 45], [56, 66]],
           [[17, 27, 37, 47], [57, 67]],
           [[18, 28, 38, 48], [58, 68]]]

CV = CrossValidation(Dataset, 1)

TR, VL = CV.GetKFold(0)

print(TR)
print("")
print(VL)

# Graph

In [ ]:
# test di training

def update_weights(bridges_weights:list,last_output,loss):
  learning_rate= 0.1
  assert learning_rate > 0 and learning_rate < 1
  new_weights= map(bridges_weights)



# ModelSelection

In [ ]:
#responsabilità: selezionare un modello, la classe è astratta e da essa poi dovranno ereditare altre classi che implemetano una strategia di validazione
class ModelSelection:
  def __init__(self,Model,TrSet,Retraining=False):
    self.__Model=Model
    self.__TrSet=TrSet
    self.__Retraining=Retraining
    # data una serie di iperparametri e un regola per selezionare i valori dalla epoch restituisce il risultato della selezione del modello
  def Select(self,EpochRule,Hyperparameters):
    raise NotImplemented


# KFoldCV(ModelSelection)

# MAIN

In [ ]:
def weights_update_function(weigths,GradientLoss,LearningRate=0.1,WeightDecay=0):
  return list(map(lambda w: w +LearningRate*(GradientLoss + WeightDecay*w),weigths)) # nota : 0.1 = eta


def BuildTwoLevelFeedForward(InputSize,HiddenSize,OutputSize):
  LossFunction= lambda yo, yt: 1/2 * sum((yo - yt)**2)
  NN=NeuralNetwork(InputSize,OutputSize,LossFunction)
  InputLayer=NN.GetAllInputNeurons()
  OutputLayer=NN.GetAllOutputNeurons()
  HiddenLayer = [ ActivationNeuron(lambda x:x,weights_update_function,lambda x,y:0,LossFunction) for _ in range(HiddenSize) ]
  for InputNeuron in InputLayer:
    for HiddenNeuron in HiddenLayer:
      InputNeuron.AddConnectionTo(HiddenNeuron)
  for OutputNeuron in OutputLayer:
    for HiddenNeuron in HiddenLayer:
      HiddenNeuron.AddConnectionTo(OutputNeuron)
  NN.CalculateAllStructure()
  return NN

TwoLevelFF=BuildTwoLevelFeedForward(10,5,3)

DataSet=np.loadtxt('sample_data/california_housing_train.csv',delimiter=',',skiprows=1,dtype=np.float64)

K=6

# codice di esempio non ancroa funzionante






In [ ]:

training=TrainPhase(TwoLevelFF,Data[:100])

for _ in range(100):
  training.Work()
  print(training.GetMetrics())
prova=dict()
prova["Training loss"]=training.GetMetrics()["Loss"]

Graph(prova,["red"],"","")

In [ ]:


def ModelSelection(Model,DataSet,folds,*hyperparams):
  ModelSelectionPerformance=dict()
  TrainingPerformance=[]
  ValidationPerformance=[]
  cv=CrossValidation(DataSet,folds)
  for fold in range(cv.GetFoldsNum()):
    tr,vl = cv.GetKFold(fold)
    training=TrainPhase(Model,tr)
    for i in range(50):# fine training dopo un numero fisso di epoche / verrà cambiato con un criterio di fermata
      training.Work()
    evaluation=EvaluationPhase(Model,vl)
    evaluation.Work()
    TrainingPerformance.append(training.GetMetrics())
    ValidationPerformance.append(evaluation.GetMetrics())
  ModelSelectionPerformance["training"]=TrainingPerformance
  ModelSelectionPerformance["validation"]=ValidationPerformance
  ModelSelectionPerformance["hyperparameters"]=dict()
  return ModelSelectionPerformance





def DisplayKFoldPerformance(ModelSelectionData):
  if(len(ModelSelectionData["training"])!=len(ModelSelectionData["training"])): raise ValueError("folds metrics bust be of the same size")
  XLen = max(max(map(len,ModelSelectionData["training"])), max(map(len,ModelSelectionData["validation"])))
  Xdata=np.linspace(0,XLen,XLen)


# hyperparameters

LearningRate=np.linspace(0.1,0.9,10)
WeightDecay=np.linspace(0,2,10)# a.k.a. Lambda in Tikohonv regularization
FoldsNumber= [6]
BatchDimension= [10]

ParameterGrid=list(it.product(LearningRate,WeightDecay,FoldsNumber,BatchDimension))

print(f"performing grid search on {len(ParameterGrid[0])} hyperparamters with {len(ParameterGrid)} combinations")

Model=TwoLevelFF
Data=Data


for hyperparameters in ParameterGrid:
  Results=ModelSelection(Model,Data,*hyperparameters)





performing grid search on 4 hyperparamters with 100 combinations
